Imports

In [ ]:
""" Librerías """

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re

sns.set_style('darkgrid')
np.set_printoptions(precision=2)
warnings.filterwarnings("ignore")

import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

from sklearn.model_selection import KFold, ShuffleSplit, LeaveOneOut, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV


""" lectura """

from google.colab import drive
drive.mount('/content/drive')

import pickle

""" forms  """

!pip install gspread
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials

""" Semilla """

seed = 42

Mounted at /content/drive


DATASET

Este es el fichero que ya esta tokenizado

In [ ]:
df_recipes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/Dataset Definitivo/PP_recipes.csv')

print(f'Número de filas: {df_recipes.shape[0]}, Número de columnas:{df_recipes.shape[1]}')

Número de filas: 178265, Número de columnas:8


In [ ]:
df_recipes.head(5)

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [ ]:
df_recipes.dtypes

id                    int64
i                     int64
name_tokens          object
ingredient_tokens    object
steps_tokens         object
techniques           object
calorie_level         int64
ingredient_ids       object
dtype: object

In [ ]:
N_Calorias = df_recipes['calorie_level'].value_counts()
print(N_Calorias)

0    69699
1    63255
2    45311
Name: calorie_level, dtype: int64


In [ ]:
df_RAW_recipes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/Dataset Definitivo/RAW_recipes.csv')


In [ ]:
df_RAW_recipes.head(5)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [ ]:
df_recipes.shape , df_RAW_recipes.shape


((178265, 8), (231637, 12))

Para poder unir df_recipes y df_RAW_recipes necesitamos tener el fichero de interacciones que es la que tiene el RAW interacciones.

In [ ]:
df_RAW_interactions = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/Dataset Definitivo/RAW_interactions.csv')

In [ ]:
df_RAW_interactions.shape

(1132367, 5)

In [ ]:
df_RAW_interactions.head(5)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [ ]:
data_merge = pd.merge(df_RAW_interactions,df_RAW_recipes, right_on='id',left_on='recipe_id')

In [ ]:
data_merge.shape

(1132367, 17)

In [ ]:
data_merge.head(10)

,user_id,recipe_id,date,rating,review,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,devilicious cookie cake delights,44394,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...","[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]",5,"['blend together cake mix , oil and eggs', 'ad...",NaN,"[""devil's food cake mix"", 'vegetable oil', 'eg...",4
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13
5,52282,120345,2005-05-21,4,very very sweet. after i waited the 2 days i b...,sugared raspberries,120345,10,37449,2005-05-02,"['15-minutes-or-less', 'time-to-make', 'course...","[838.0, 1.0, 820.0, 0.0, 2.0, 0.0, 71.0]",6,"['carefully pick over the berries , removing l...",here's an old method for preserving fruit with...,"['raspberries', 'granulated sugar']",2
6,124416,120345,2011-08-06,0,"Just an observation, so I will not rate. I fo...",sugared raspberries,120345,10,37449,2005-05-02,"['15-minutes-or-less', 'time-to-make', 'course...","[838.0, 1.0, 820.0, 0.0, 2.0, 0.0, 71.0]",6,"['carefully pick over the berries , removing l...",here's an old method for preserving fruit with...,"['raspberries', 'granulated sugar']",2
7,2000192946,120345,2015-05-10,2,This recipe was OVERLY too sweet. I would sta...,sugared raspberries,120345,10,37449,2005-05-02,"['15-minutes-or-less', 'time-to-make', 'course...","[838.0, 1.0, 820.0, 0.0, 2.0, 0.0, 71.0]",6,"['carefully pick over the berries , removing l...",here's an old method for preserving fruit with...,"['raspberries', 'granulated sugar']",2
8,76535,134728,2005-09-02,4,Very good!,kfc honey bbq strips,134728,40,166019,2005-08-24,"['60-minutes-or-less', 'time-to-make', 'main-i...","[316.0, 4.0, 40.0, 37.0, 78.0, 4.0, 10.0]",10,"['mix flour , salt and pepper in bowl', 'set a...",these are so yummy and they do taste just like...,"['chicken tenders', 'flour', 'garlic powder', ...",12
9,273745,134728,2005-12-22,5,Better than the real!!,kfc honey bbq strips,134728,40,166019,2005-08-24,"['60-minutes-or-less', 'time-to-make', 'main-i...","[316.0, 4.0, 40.0, 37.0, 78.0, 4.0, 10.0]",10,"['mix flour , salt and pepper in bowl', 'set a...",these are so yummy and they do taste just like...,"['chicken tenders', 'flour', 'garlic powder', ...",12


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/Dataset Definitivo/RAW_recipes.csv')

Se usa la información de la columna nutrition para separar los datos de calorias, sodio, proteinas, etc

In [ ]:
data[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']] = data.nutrition.str.split(",",expand=True)
data['calories'] = data['calories'].apply(lambda x: x.replace("[" ,""))
data['carbohydrates'] = data['carbohydrates'].apply(lambda x: x.replace("]" ,""))
data[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']] =  data[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']].astype(float)

In [ ]:
data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


In [ ]:
data.dtypes

name               object
id                  int64
minutes             int64
contributor_id      int64
submitted          object
tags               object
nutrition          object
n_steps             int64
steps              object
description        object
ingredients        object
n_ingredients       int64
calories          float64
total fat         float64
sugar             float64
sodium            float64
protein           float64
saturated fat     float64
carbohydrates     float64
dtype: object

Validamos el minimo y maximo de calorias

In [ ]:
Calorias = data['calories'].value_counts()
print(Calorias)

105.0     70
226.2     69
143.2     68
156.2     68
104.0     67
          ..
2068.3     1
2029.9     1
2072.8     1
1837.3     1
2641.4     1
Name: calories, Length: 21479, dtype: int64


In [ ]:
#data['calories'] = data['calories'].astype(int)

# Ahora, visualizamos el histograma
#plt.hist(data['calories'], bins=50)  # El número de bins (barras) puede ser ajustado según tus preferencias
#plt.xlabel('calories')
#plt.ylabel('Frecuencia')
#plt.title('Histograma de Calorias')
#plt.show()

In [ ]:
Calorias_Max = data['calories'].max()
Calorias_Min = data['calories'].min()
Calorias_Med = data['calories'].mean()
print(Calorias_Max), print(Calorias_Min), print(Calorias_Med)


434360.2
0.0
473.9424245694772


(None, None, None)

In [ ]:
# Definir los límites para las categorías de calorías
calorias_low_limit = data['calories'].quantile(1/3)
calorias_high_limit = data['calories'].quantile(2/3)

# Función para asignar la categoría a cada valor de calorías
def assign_calorias_level(calorias):
    if calorias <= calorias_low_limit:
        return 0
    elif calorias <= calorias_high_limit:
        return 1
    else:
        return 2

# Crear la nueva columna "calorias_level" aplicando la función a la columna "calories"
data['calorias_level'] = data['calories'].apply(assign_calorias_level)

# Verificar el balance de las categorías
calorias_counts = data['calorias_level'].value_counts()
print("Balance de categorías:")
print(calorias_counts)


Balance de categorías:
0    77236
1    77207
2    77194
Name: calorias_level, dtype: int64


In [ ]:
cantidad_calorias = len(data['calories'])
print(cantidad_calorias)

231637


In [ ]:
data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates,calorias_level
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,1
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,1
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,1


Creamos nueva columna para poder agregar el tipo de comida

In [ ]:
data['food types'] = np.nan
data['food types'] = data['food types'].astype('str')

In [ ]:
for i in data['ingredients'].index:
    if('eggs' not in data['ingredients'][i]):
         if('ice-cream' in data['ingredients'][i] or 'chocolate' in data['ingredients'][i] or 'cookies' in data['ingredients'][i]):
                data['food types'][i]='Veg dessert'
    elif('eggs' in data['ingredients'][i]):
        if('ice-cream' in data['ingredients'][i] or 'chocolate' in data['ingredients'][i] or 'cookies' in data['ingredients'][i]):
                data['food types'][i]='Non-Veg dessert'

In [ ]:
for i in data.index:
    if(data['food types'][i]!='Veg dessert' and data['food types'][i]!='Non-Veg dessert' and 20<data['calories'][i]<300):
        data['food types'][i]='Healthy'

In [ ]:
for i in data.index:
    if(data['food types'][i]!='Veg dessert' and data['food types'][i]!='Non-Veg dessert' and data['food types'][i]!='Healthy'):
        if('chicken' in data['ingredients'][i] or 'eggs' in data['ingredients'][i] or'ham' in data['ingredients'][i] or 'pepperoni' in data['ingredients'][i] ):
            data['food types'][i]='Non-veg'



In [ ]:
for i in data.index:
    if(data['food types'][i]!='Veg dessert' and data['food types'][i]!='Non-Veg dessert' and data['food types'][i]!='Healthy' and data['food types'][i]!='Non-veg'):
        data['food types'][i]='Veg'


In [ ]:
data['food types'].value_counts()

Healthy            100579
Veg                 70416
Non-veg             45948
Veg dessert          9532
Non-Veg dessert      5162
Name: food types, dtype: int64

In [ ]:
data['food types'].isnull().sum()

0

In [ ]:
data.head()


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,...,n_ingredients,calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates,calorias_level,food types
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,...,7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,0,Healthy
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,...,6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,0,Healthy
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,...,13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,1,Healthy
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...",...,11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,1,Veg
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,...,8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,1,Veg


Prueba del fichero pkl que está en kaggle

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/TFM/Dataset Definitivo/ingr_map.pkl'

# Leer el archivo .pkl
with open(file_path, 'rb') as file:
    data_pk = pickle.load(file)

In [ ]:
data_pk.shape

(11659, 7)

In [ ]:
data_pk.dtypes

raw_ingr     object
raw_words     int64
processed    object
len_proc      int64
replaced     object
count         int64
id            int16
dtype: object

In [ ]:
data_pk.head(20)

,raw_ingr,raw_words,processed,len_proc,replaced,count,id
0,"medium heads bibb or red leaf lettuce, washed,...",13,"medium heads bibb or red leaf lettuce, washed,...",73,lettuce,4507,4308
1,mixed baby lettuces and spring greens,6,mixed baby lettuces and spring green,36,lettuce,4507,4308
2,romaine lettuce leaf,3,romaine lettuce leaf,20,lettuce,4507,4308
3,iceberg lettuce leaf,3,iceberg lettuce leaf,20,lettuce,4507,4308
4,red romaine lettuce,3,red romaine lettuce,19,lettuce,4507,4308
5,head romaine lettuce,3,head romaine lettuce,20,lettuce,4507,4308
6,curly endive lettuce,3,curly endive lettuce,20,lettuce,4507,4308
7,romaine lettuce hearts,3,romaine lettuce heart,21,lettuce,4507,4308
8,baby leaf lettuce,3,baby leaf lettuce,17,lettuce,4507,4308
9,head of lettuce,3,head of lettuce,15,lettuce,4507,4308


Hacemos un merge

In [ ]:
recipes = df_recipes.merge(right=df_RAW_recipes, left_on="id", right_on="id")
recipes = recipes[["id", "name", "submitted", "ingredient_ids", "ingredients", "n_ingredients"]]
recipes = recipes.rename({"id":"recipe_id"}, axis="columns")
recipes.head()

,recipe_id,name,submitted,ingredient_ids,ingredients,n_ingredients
0,424415,aromatic basmati rice rice cooker,2010-05-10,"[389, 7655, 6270, 1527, 3406]","['basmati rice', 'water', 'salt', 'cinnamon st...",5
1,146223,pumpkin pie a la easy,2005-11-25,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,...","['flour', 'oats', 'brown sugar', 'pecans', 'bu...",12
2,312329,cheesy tomato soup with potatoes,2008-07-07,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696...","['chicken broth', 'water', 'salt', 'black pepp...",15
3,74301,mini tacos,2003-10-29,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]","['wonton wrappers', 'hamburger', 'taco seasoni...",8
4,76272,rosemary s hanky panky s,2003-11-12,"[3484, 6324, 7594, 243]","['ground beef', 'ground sausage', 'velveeta ch...",4


In [ ]:
df_RAW_interactions.head()

interactions = df_RAW_interactions[["user_id", "recipe_id", "rating", "review"]]
interactions.head()



,user_id,recipe_id,rating,review
0,38094,40893,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,4,This worked very well and is EASY. I used not...
3,126440,85009,5,I made the Mexican topping and took it to bunk...
4,57222,85009,5,"Made the cheddar bacon topping, adding a sprin..."


In [ ]:
ingr_map = data_pk

In [ ]:
ingr_df = ingr_map.copy(deep=True)
ingr_df = ingr_df.rename({"id":"ingr_id","replaced":"ingr_name"}, axis="columns")
ingr_df = ingr_df[["ingr_id", "ingr_name"]]
ingr_df = ingr_df.drop_duplicates(ignore_index=True)
ingr_df.head(10)


,ingr_id,ingr_name
0,4308,lettuce
1,2744,french vanilla pudding and pie filling mix
2,6843,stove top stuffing mix
3,1910,cream cheese
4,1168,cheddar
5,5811,radicchio
6,5205,pasta sauce
7,7233,tomato sauce
8,6482,shredded three cheese
9,4836,mozzarella


Tokenizing

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
words = set(nltk.corpus.words.words())

In [ ]:

lemmatizer = WordNetLemmatizer()

def nltk_pos_tagger(nltk_tag):
    if nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    else:
        return None

def tokenize_sentence(sentence):
    tokenizer = nltk.RegexpTokenizer(r"[^\d\W]+")
    tokenized = tokenizer.tokenize(sentence)
    stopwords = nltk.corpus.stopwords.words('english')
    finalsentence = [word for word in tokenized if word not in stopwords]
    return(finalsentence)

def lemmatize_sentence(sentence):

    nltk_tagged = nltk.pos_tag(sentence)
    wordnet_tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])), nltk_tagged)
    lemmatized_sentence = []

    for word, tag in wordnet_tagged:
        if not (tag is None):
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return (lemmatized_sentence)

def tokenize_lemmatize(sentence):
  tokenized = tokenize_sentence(sentence)
  lemmatized = lemmatize_sentence(tokenized)
  selectedwords = [word for word in lemmatized if word in words]
  final = list(dict.fromkeys(selectedwords))
  return(final)



In [ ]:
data['recsys'] = data.apply(lambda row: tokenize_lemmatize(str(row['name']) + str(row['steps']) + str(row['ingredients'])), axis=1)




In [ ]:
# Create a function to tokenize and lemmatize a single string
def tokenize_lemmatize(sentence):
    tokenized = tokenize_sentence(sentence)
    lemmatized = lemmatize_sentence(tokenized)
    selectedwords = [word for word in lemmatized if word in words]
    final = list(dict.fromkeys(selectedwords))
    return final

# Tokenize and lemmatize each column separately and store the results in new columns
data['name_tokens'] = data['name'].apply(lambda x: tokenize_lemmatize(str(x)))
data['steps_tokens'] = data['steps'].apply(lambda x: tokenize_lemmatize(str(x)))
data['ingredients_tokens'] = data['ingredients'].apply(lambda x: tokenize_lemmatize(str(x)))

In [ ]:
data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,...,sodium,protein,saturated fat,carbohydrates,calorias_level,food types,recsys,name_tokens,steps_tokens,ingredients_tokens
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,...,0.0,2.0,0.0,4.0,0,Healthy,"[arriba, bake, winter, squash, style, make, ch...","[arriba, bake, winter, squash, style]","[make, choice, proceed, recipe, depend, size, ...","[winter, squash, seasoning, spice, honey, butt..."
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,...,17.0,22.0,35.0,1.0,0,Healthy,"[bit, breakfast, pizza, preheat, degree, f, pr...","[bit, breakfast, pizza]","[preheat, press, bottom, side, inch, bake, min...","[pizza, crust, sausage, patty, egg, milk, salt..."
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,...,48.0,39.0,27.0,5.0,1,Healthy,"[kitchen, chili, ground, beef, pot, add, chop,...","[kitchen, chili]","[ground, beef, pot, add, chop, onion, wilt, in...","[ground, beef, onion, dice, tomato, paste, sou..."
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...",...,2.0,14.0,8.0,20.0,1,Veg,"[potato, place, pot, salt, water, bring, boil,...",[potato],"[place, potato, pot, salt, water, bring, boil,...","[potato, shallot, parsley, oil, wine, vinegar,..."
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,...,23.0,3.0,0.0,28.0,1,Veg,"[tomato, ketchup, can, ingredient, boil, hour,...","[tomato, ketchup, canning]","[mix, ingredient, boil, hour, thick, pour, jar...","[tomato, juice, apple, cider, vinegar, sugar, ..."


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a new instance of LabelEncoder
label_encoder = LabelEncoder()

# List of columns to tokenize and encode
columns_to_tokenize = ['name_tokens', 'ingredients_tokens', 'steps_tokens']

# Loop through each column and apply tokenization and encoding
for column in columns_to_tokenize:
    data[f'{column}_ids'] = data[column].apply(lambda x: label_encoder.fit_transform(x))



In [ ]:
data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,...,carbohydrates,calorias_level,food types,recsys,name_tokens,steps_tokens,ingredients_tokens,name_tokens_ids,ingredients_tokens_ids,steps_tokens_ids
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,...,4.0,0,Healthy,"[arriba, bake, winter, squash, style, make, ch...","[arriba, bake, winter, squash, style]","[make, choice, proceed, recipe, depend, size, ...","[winter, squash, seasoning, spice, honey, butt...","[0, 1, 4, 2, 3]","[7, 6, 4, 5, 1, 0, 2, 3]","[19, 5, 25, 26, 9, 31, 35, 7, 15, 27, 30, 34, ..."
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,...,1.0,0,Healthy,"[bit, breakfast, pizza, preheat, degree, f, pr...","[bit, breakfast, pizza]","[preheat, press, bottom, side, inch, bake, min...","[pizza, crust, sausage, patty, egg, milk, salt...","[0, 1, 2]","[5, 1, 7, 4, 2, 3, 6, 0]","[13, 14, 1, 17, 6, 0, 8, 16, 3, 15, 11, 21, 5,..."
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,...,5.0,1,Healthy,"[kitchen, chili, ground, beef, pot, add, chop,...","[kitchen, chili]","[ground, beef, pot, add, chop, onion, wilt, in...","[ground, beef, onion, dice, tomato, paste, sou...","[1, 0]","[6, 1, 9, 5, 14, 10, 13, 7, 0, 15, 3, 11, 4, 1...","[7, 2, 12, 0, 4, 11, 15, 9, 1, 3, 5, 6, 8, 13,..."
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...",...,20.0,1,Veg,"[potato, place, pot, salt, water, bring, boil,...",[potato],"[place, potato, pot, salt, water, bring, boil,...","[potato, shallot, parsley, oil, wine, vinegar,...",[0],"[3, 5, 2, 1, 7, 6, 4, 0, 8]","[20, 23, 22, 27, 33, 5, 3, 7, 32, 10, 4, 0, 15..."
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,...,28.0,1,Veg,"[tomato, ketchup, can, ingredient, boil, hour,...","[tomato, ketchup, canning]","[mix, ingredient, boil, hour, thick, pour, jar...","[tomato, juice, apple, cider, vinegar, sugar, ...","[2, 1, 0]","[8, 3, 0, 1, 9, 7, 6, 2, 5, 4]","[10, 4, 0, 3, 15, 12, 5, 16, 2, 6, 1, 11, 7, 9..."


Creación de las tecnicas

In [ ]:

# Lista de técnicas de cocina que queremos buscar (incluyendo formas verbales en infinitivo)
#tecnicas_cocina = ['baking', 'bake', 'grilling', 'grill', 'broiling', 'broil', 'boiling', 'boil', 'sauteing', 'saute', 'roasting', 'roast', 'frying', 'fry', 'steaming', 'steam', 'poaching', 'poach', 'simmering', 'simmer', 'stir-frying', 'stir-fry', 'blanching', 'blanch', 'braising', 'braise', 'marinating', 'marinate', 'smoking', 'smoke']
tecnicas_cocina = [ 'bake', 'grill', 'boil', 'saute', 'roast',  'fry',  'steam', 'simmer',  'blanch', 'braise', 'marinate', 'smoke', 'blend', 'brown', 'cook', 'boiler']

# Función para encontrar las técnicas de cocina en un texto
def encontrar_tecnicas(texto):
    tecnicas_encontradas = []
    for tecnica in tecnicas_cocina:
        #if re.search(r'\b' + tecnica + r'\b', texto, re.IGNORECASE):     # Aplica solo si vamos a usar texto
            tecnicas_encontradas.append(tecnica)
    return tecnicas_encontradas

# Crear una nueva columna 'tecnicas_cocina' que contenga las técnicas encontradas en cada fila
data['tecnicas_cocina'] = data['recsys'].apply(encontrar_tecnicas)

# Mostrar las primeras filas del DataFrame con la nueva columna
print(data.head())




NameError: ignored

Vertorización en una sola columna

In [ ]:
# Función para encontrar las técnicas de cocina en una lista de tokens
def encontrar_tecnicas(tokens):
    tecnicas_encontradas = []
    for tecnica in tecnicas_cocina:
        if tecnica in tokens:
            tecnicas_encontradas.append(tecnica)
    return tecnicas_encontradas

# Crear una nueva columna 'tecnicas_cocina' que contenga las técnicas encontradas en cada fila
data['tecnicas_cocina'] = data['steps_tokens'].apply(encontrar_tecnicas)

# Mostrar las primeras filas del DataFrame con la nueva columna
print(data.head())

KeyError: ignored

vectorizacion

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Convert the list of techniques in each row into a space-separated string
data['tecnicas_cocina_str'] = data['tecnicas_cocina'].apply(lambda x: ' '.join(x))

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the vectorizer on the 'tecnicas_cocina_str' column
tecnicas_vectorized = vectorizer.fit_transform(data['tecnicas_cocina_str'])

# Create a new column 'techniques' with the vectorized techniques
data['techniques'] = list(tecnicas_vectorized.toarray())

# Drop the intermediate 'tecnicas_cocina_str' column if needed
data.drop(columns=['tecnicas_cocina_str'], inplace=True)

# Show the DataFrame with the vectorized techniques
print(data.head())

                                         name      id  minutes  \
0  arriba   baked winter squash mexican style  137739       55   
1            a bit different  breakfast pizza   31490       30   
2                   all in the kitchen  chili  112140      130   
3                          alouette  potatoes   59389       45   
4          amish  tomato ketchup  for canning   44061      190   

   contributor_id   submitted  \
0           47892  2005-09-16   
1           26278  2002-06-17   
2          196586  2005-02-25   
3           68585  2003-04-14   
4           41706  2002-10-25   

                                                tags  \
0  ['60-minutes-or-less', 'time-to-make', 'course...   
1  ['30-minutes-or-less', 'time-to-make', 'course...   
2  ['time-to-make', 'course', 'preparation', 'mai...   
3  ['60-minutes-or-less', 'time-to-make', 'course...   
4  ['weeknight', 'time-to-make', 'course', 'main-...   

                                    nutrition  n_steps  \
0       [

In [ ]:
data.head(25)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,...,food types,recsys,name_tokens,steps_tokens,ingredients_tokens,name_tokens_ids,ingredients_tokens_ids,steps_tokens_ids,tecnicas_cocina,techniques
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,...,Healthy,"[arriba, bake, winter, squash, style, make, ch...","[arriba, bake, winter, squash, style]","[make, choice, proceed, recipe, depend, size, ...","[winter, squash, seasoning, spice, honey, butt...","[0, 1, 4, 2, 3]","[7, 6, 4, 5, 1, 0, 2, 3]","[19, 5, 25, 26, 9, 31, 35, 7, 15, 27, 30, 34, ...",[bake],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,...,Healthy,"[bit, breakfast, pizza, preheat, degree, f, pr...","[bit, breakfast, pizza]","[preheat, press, bottom, side, inch, bake, min...","[pizza, crust, sausage, patty, egg, milk, salt...","[0, 1, 2]","[5, 1, 7, 4, 2, 3, 6, 0]","[13, 14, 1, 17, 6, 0, 8, 16, 3, 15, 11, 21, 5,...","[bake, brown]","[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,...,Healthy,"[kitchen, chili, ground, beef, pot, add, chop,...","[kitchen, chili]","[ground, beef, pot, add, chop, onion, wilt, in...","[ground, beef, onion, dice, tomato, paste, sou...","[1, 0]","[6, 1, 9, 5, 14, 10, 13, 7, 0, 15, 3, 11, 4, 1...","[7, 2, 12, 0, 4, 11, 15, 9, 1, 3, 5, 6, 8, 13,...",[cook],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...",...,Veg,"[potato, place, pot, salt, water, bring, boil,...",[potato],"[place, potato, pot, salt, water, bring, boil,...","[potato, shallot, parsley, oil, wine, vinegar,...",[0],"[3, 5, 2, 1, 7, 6, 4, 0, 8]","[20, 23, 22, 27, 33, 5, 3, 7, 32, 10, 4, 0, 15...","[bake, boil, cook]","[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,...,Veg,"[tomato, ketchup, can, ingredient, boil, hour,...","[tomato, ketchup, canning]","[mix, ingredient, boil, hour, thick, pour, jar...","[tomato, juice, apple, cider, vinegar, sugar, ...","[2, 1, 0]","[8, 3, 0, 1, 9, 7, 6, 2, 5, 4]","[10, 4, 0, 3, 15, 12, 5, 16, 2, 6, 1, 11, 7, 9...",[boil],"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,apple a day milk shake,5289,0,1533,1999-12-06,"['15-minutes-or-less', 'time-to-make', 'course...","[160.2, 10.0, 55.0, 3.0, 9.0, 20.0, 7.0]",4,"['combine ingredients in blender', 'cover and ...",NaN,...,Healthy,"[apple, day, milk, ingredient, blender, blend,...","[apple, day, milk, shake]","[combine, ingredient, blender, blend, sprinkle...","[milk, vanilla, ice, cream, frozen, apple, jui...","[0, 1, 2, 3]","[6, 7, 4, 2, 3, 0, 5, 1]","[3, 6, 1, 0, 8, 5, 2, 7, 4]",[blend],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6,aww marinated olives,25274,15,21730,2002-04-14,"['15-minutes-or-less', 'time-to-make', 'course...","[380.7, 53.0, 7.0, 24.0, 6.0, 24.0, 6.0]",4,['toast the fennel seeds and lightly crush the...,m

In [ ]:
data['tecnicas_cocina_tuplas'] = data['tecnicas_cocina'].apply(tuple)

# Obtener las categorías únicas
categorias_unicas = data['tecnicas_cocina_tuplas'].unique()

# Imprimir las categorías únicas
print(categorias_unicas)

[('bake',) ('bake', 'brown') ('cook',) ...
 ('bake', 'saute', 'blanch', 'cook')
 ('bake', 'grill', 'roast', 'steam', 'marinate')
 ('grill', 'saute', 'fry', 'smoke', 'cook')]


In [ ]:
data.dtypes

name                       object
id                          int64
minutes                     int64
contributor_id              int64
submitted                  object
tags                       object
nutrition                  object
n_steps                     int64
steps                      object
description                object
ingredients                object
n_ingredients               int64
calories                  float64
total fat                 float64
sugar                     float64
sodium                    float64
protein                   float64
saturated fat             float64
carbohydrates             float64
calorias_level              int64
food types                 object
recsys                     object
name_tokens                object
steps_tokens               object
ingredients_tokens         object
name_tokens_ids            object
ingredients_tokens_ids     object
steps_tokens_ids           object
tecnicas_cocina            object
techniques    

Descarga del fichero a cvs.

In [ ]:

# Especifica la ruta y el nombre de archivo donde deseas guardar el CSV
ruta_archivo_csv = '/content/drive/MyDrive/Colab Notebooks/TFM/Dataset Definitivo/data_transformado_f.csv'

# Utiliza el método to_csv() para guardar el DataFrame en formato CSV
data.to_csv(ruta_archivo_csv, index=False)

# Imprime un mensaje de confirmación
print("El DataFrame ha sido guardado exitosamente como CSV.")

El DataFrame ha sido guardado exitosamente como CSV.


In [ ]:

import matplotlib.pyplot as plt
from wordcloud import WordCloud

ingred = data['steps'].str.split(expand=True).stack().reset_index(level=0).rename(columns={0: 'steps'})
ingred['steps'] = ingred['steps'].str.replace('"', '')
ingred = ingred.groupby('steps').size().reset_index(name='n')
ingred = ingred[~ingred['steps'].isin(['s', 'the', 'and', 'or', 'a', ''])]

wordcloud = WordCloud(width=800, height=400, max_words=200, background_color='white', colormap='viridis').generate_from_frequencies(dict(ingred.values))
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
pd.set_option('display.max_colwidth', None)  # Ver ejemplo de los pasos para preparar la receta
print(data['steps'].head(5))

0    ['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']
1                                                                                                                                                                                                                                                                      

In [ ]:
data.drop(["submitted","contributor_id"],axis=1,inplace=True)